In [ ]:
# !unzip ./data.zip

In [1]:
!nvidia-smi

Sun Apr  4 14:16:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.39       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Graphics Device     Off  | 00000000:26:00.0  On |                  N/A |
| 59%   47C    P5    11W / 170W |    349MiB / 12045MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Check the data

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import os

In [3]:
len(os.listdir('./train'))

36

In [4]:
how_many_imgs = int(input('How many imgs ? : '))
for _ in range(how_many_imgs):
  valid_or_test_or_train = random.choice(['train','test','validation'])
  print(valid_or_test_or_train)
  random_class = random.choice(os.listdir(f'./{valid_or_test_or_train}/'))
  print(random_class)
  random_img = random.choice(os.listdir(f'./{valid_or_test_or_train}/{random_class}/'))
  print(random_img)
  path = f"./{valid_or_test_or_train}/{random_class}/{random_img}"
  print(path)
  img = mpimg.imread(path)
  img = img / 255.0
  plt.figure(figsize=(10,7))
  plt.imshow(img)
  plt.title(f'Shape : {img.shape} | Path : {path}')
  plt.axis(False)
  plt.show();

How many imgs ? : 


ValueError: ignored

In [ ]:
paths_walkthrough = ['train','test','validation']
for path_walkthrough in paths_walkthrough:
  print('\n'*5)
  print('*-*'*50)
  print(path_walkthrough)
  print('*-*'*50)
  print('\n'*5)
  for (root,dirs,files) in os.walk(path_walkthrough): 
    print('-'*50)
    print(f'Root : {root}')
    print(f'Dirs : {dirs}')
    print(f'No of Imgs : {len(files)}')
    print('-'*50)

# Loading the data

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
train_datagen = ImageDataGenerator(rescale=1/255.0,width_shift_range=0.25,height_shift_range=0.25,shear_range=0.25,zoom_range=0.25,horizontal_flip=True,vertical_flip=True,validation_split=0.85)
test_datagen = ImageDataGenerator(rescale=1/255.0,width_shift_range=0.25,height_shift_range=0.25,shear_range=0.25,zoom_range=0.25,horizontal_flip=True,vertical_flip=True)
valid_datagen = ImageDataGenerator(rescale=1/255.0,width_shift_range=0.25,height_shift_range=0.25,shear_range=0.25,zoom_range=0.25,horizontal_flip=True,vertical_flip=True)

In [7]:
test_data = test_datagen.flow_from_directory('./test',target_size=(224,224))
valid_data = valid_datagen.flow_from_directory('./validation',target_size=(224,224))
train_data = train_datagen.flow_from_directory('./train',target_size=(224,224))
train_data_small = train_datagen.flow_from_directory('./train',target_size=(224,224),subset='training')

Found 359 images belonging to 36 classes.
Found 351 images belonging to 36 classes.
Found 3581 images belonging to 36 classes.
Found 540 images belonging to 36 classes.


# Modelling

In [8]:
project_name = "Fruit-and-Vegetables-Recognition"

In [9]:
!pip3 install wandb
import wandb
from wandb.keras import WandbCallback
import tensorflow as tf
import tensorflow_hub as hub

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


## Classification vs CNN (Custom) vs Transfer Learning (resnet50)

### Classification

In [10]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session =tf.compat.v1.InteractiveSession(config=config)

In [11]:
gpu = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [12]:
# wandb.init(project=project_name,name='Classification-baseline')
# model = tf.keras.Sequential([
#   tf.keras.layers.Flatten(input_shape=(224,224,3)),
#   tf.keras.layers.Dense(4,activation='relu'),
#   tf.keras.layers.Dense(4,activation='relu'),
#   tf.keras.layers.Dense(len(os.listdir('./test')),activation='softmax')
# ])
# model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'],optimizer=tf.keras.optimizers.Adam())
# model.fit(train_data_small,validation_data=(valid_data),epochs=25,callbacks=[WandbCallback()])

In [13]:
!nvidia-smi

Sun Apr  4 14:16:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.39       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Graphics Device     Off  | 00000000:26:00.0  On |                  N/A |
| 57%   48C    P2    21W / 170W |    668MiB / 12045MiB |     15%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [14]:
# # Clean my GPU memory
# import os

# pid = os.getpid()
# !kill -9 $pid

In [15]:
!nvidia-smi

Sun Apr  4 14:16:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.39       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Graphics Device     Off  | 00000000:26:00.0  On |                  N/A |
| 57%   48C    P2    24W / 170W |    680MiB / 12045MiB |     15%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### CNN (Custom)

In [16]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session =tf.compat.v1.InteractiveSession(config=config)

/home/indika/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py:1761: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [17]:
gpu = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [18]:
# wandb.init(project=project_name,name='CNN-Custom--baseline')
# model = tf.keras.Sequential([
#   tf.keras.layers.Conv2D(10,3,activation='relu',input_shape=(224,224,3)),
#   tf.keras.layers.MaxPool2D(),
#   tf.keras.layers.Conv2D(10,3,activation='relu'),
#   tf.keras.layers.MaxPool2D(),
#   tf.keras.layers.Conv2D(10,3,activation='relu'),
#   tf.keras.layers.MaxPool2D(),
#   tf.keras.layers.Flatten(),
#   tf.keras.layers.Dense(len(os.listdir('./test')),activation='softmax')
# ])
# model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'],optimizer=tf.keras.optimizers.Adam())
# model.fit(train_data_small,validation_data=(valid_data),epochs=25,callbacks=[WandbCallback()])

In [19]:
!nvidia-smi

Sun Apr  4 14:16:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.39       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Graphics Device     Off  | 00000000:26:00.0  On |                  N/A |
| 57%   49C    P2    29W / 170W |    680MiB / 12045MiB |     12%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [20]:
## Clean my GPU memory
# import os
 
# pid = os.getpid()
# !kill -9 $pid

In [21]:
!nvidia-smi

Sun Apr  4 14:16:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.39       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Graphics Device     Off  | 00000000:26:00.0  On |                  N/A |
| 57%   49C    P2    33W / 170W |    680MiB / 12045MiB |     13%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Transfer Learning (resnet50)

In [22]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session =tf.compat.v1.InteractiveSession(config=config)
gpu = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [23]:
# wandb.init(project=project_name,name='Transfer-Learning-baseline')
# model_url = "https://tfhub.dev/tensorflow/resnet_50/feature_vector/1"
# tl_layer = hub.KerasLayer(model_url,input_shape=(224,224,3),trainable=False)
# model = tf.keras.Sequential([
#   tl_layer,
#   tf.keras.layers.Dense(len(os.listdir('./test')),activation='softmax')
# ])
# model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'],optimizer=tf.keras.optimizers.Adam())
# model.fit(train_data_small,validation_data=(valid_data),epochs=25,callbacks=[WandbCallback()])

In [24]:
!nvidia-smi

Sun Apr  4 14:16:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.39       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Graphics Device     Off  | 00000000:26:00.0  On |                  N/A |
| 57%   49C    P2    35W / 170W |    680MiB / 12045MiB |     21%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [25]:
## Clean my GPU memory
# import os
 
# pid = os.getpid()
# !kill -9 $pid

**Who won ???** *Transfer Learning (resnet50) or CNN (Custom) or Classification*

<hr>

#### `Transfer Learning` Won !

# What is the best `optimizer` for my project ? 

- Adadelta
- Adagrad
- Adam
- Adamax
- SGD

In [26]:
from tensorflow.keras.optimizers import *

In [27]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session =tf.compat.v1.InteractiveSession(config=config)
gpu = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [28]:
# optimizers = [Adadelta,Adadelta,Adam,Adamax,SGD]
# optimizers = [Adam,Adamax,SGD]
# for optimizer in optimizers:
#   wandb.init(project=project_name,name=f'Optimizer-{optimizer}')
#   model_url = "https://tfhub.dev/tensorflow/resnet_50/feature_vector/1"
#   tl_layer = hub.KerasLayer(model_url,input_shape=(224,224,3),trainable=False)
#   model = tf.keras.Sequential([
#     tl_layer,
#     tf.keras.layers.Dense(len(os.listdir('./test')),activation='softmax')
#   ])
#   model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'],optimizer=optimizer())
#   model.fit(train_data_small,validation_data=(valid_data),epochs=25,callbacks=[WandbCallback()])

In [29]:
!nvidia-smi

Sun Apr  4 14:16:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.39       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Graphics Device     Off  | 00000000:26:00.0  On |                  N/A |
| 57%   49C    P2    35W / 170W |    680MiB / 12045MiB |     15%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Adam optimizer won the test**

# `lr` testing

In [30]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session =tf.compat.v1.InteractiveSession(config=config)
gpu = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [31]:
# lrs = [1.0,0.1,0.01,0.001,0.0001]
# for lr in lrs:
#   wandb.init(project=project_name,name=f'lr-{lr}')
#   model_url = "https://tfhub.dev/tensorflow/resnet_50/feature_vector/1"
#   tl_layer = hub.KerasLayer(model_url,input_shape=(224,224,3),trainable=False)
#   model = tf.keras.Sequential([
#     tl_layer,
#     tf.keras.layers.Dense(len(os.listdir('./test')),activation='softmax')
#   ])
#   model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'],optimizer=Adam(lr=lr))
#   model.fit(train_data_small,validation_data=(valid_data),epochs=25,callbacks=[WandbCallback()])
#   wandb.finish()

In [32]:
!nvidia-smi

Sun Apr  4 14:17:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.39       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Graphics Device     Off  | 00000000:26:00.0  On |                  N/A |
| 57%   49C    P2    35W / 170W |    678MiB / 12045MiB |     13%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Testing

In [33]:
# wandb.init(project=project_name,name='testing', 
#            config={"architecture": "ResNet",
#                    "epochs": 37,
#                    "batch_size": 32,
#                    "iterations": 37,
#                    "learning_rate": 0.001,
#                    "features": len(os.listdir('./test'))})
# model_url = "https://tfhub.dev/tensorflow/resnet_50/feature_vector/1"
# tl_layer = hub.KerasLayer(model_url,input_shape=(224,224,3),trainable=False)
# model = tf.keras.Sequential([
#   tl_layer,
#   tf.keras.layers.Dense(len(os.listdir('./test')),activation='softmax')
# ])
# model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'],optimizer=tf.keras.optimizers.Adam())
# model.fit(train_data_small,validation_data=(valid_data),epochs=37,callbacks=[WandbCallback()])
# wandb.finish()

# What is the best transfer learning model ? 

In [34]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session =tf.compat.v1.InteractiveSession(config=config)
gpu = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [35]:
## Clean my GPU memory
# import os
 
# pid = os.getpid()
# !kill -9 $pid

In [36]:
# wandb.init(project=project_name,name='efficentnet-b1')
# model_url = "https://tfhub.dev/tensorflow/efficientnet/b1/feature-vector/1"
# tl_layer = hub.KerasLayer(model_url,input_shape=(224,224,3),trainable=False)
# model = tf.keras.Sequential([
#   tl_layer,
#   tf.keras.layers.Dense(len(os.listdir('./test')),activation='softmax')
# ])
# model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'],optimizer=tf.keras.optimizers.Adam())
# model.fit(train_data_small,validation_data=(valid_data),epochs=25,callbacks=[WandbCallback()])

In [37]:
# wandb.init(project=project_name,name='efficentnet-b2')
# model_url = "https://tfhub.dev/tensorflow/efficientnet/b2/feature-vector/1"
# tl_layer = hub.KerasLayer(model_url,input_shape=(224,224,3),trainable=False)
# model = tf.keras.Sequential([
#   tl_layer,
#   tf.keras.layers.Dense(len(os.listdir('./test')),activation='softmax')
# ])
# model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'],optimizer=tf.keras.optimizers.Adam())
# model.fit(train_data_small,validation_data=(valid_data),epochs=25,callbacks=[WandbCallback()])

In [38]:
# wandb.init(project=project_name,name='efficentnet-b3')
# model_url = "https://tfhub.dev/tensorflow/efficientnet/b3/feature-vector/1"
# tl_layer = hub.KerasLayer(model_url,input_shape=(224,224,3),trainable=False)
# model = tf.keras.Sequential([
#   tl_layer,
#   tf.keras.layers.Dense(len(os.listdir('./test')),activation='softmax')
# ])
# model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'],optimizer=tf.keras.optimizers.Adam())
# model.fit(train_data_small,validation_data=(valid_data),epochs=25,callbacks=[WandbCallback()])

In [39]:
# wandb.init(project=project_name,name='efficentnet-b4')
# model_url = "https://tfhub.dev/tensorflow/efficientnet/b4/feature-vector/1"
# tl_layer = hub.KerasLayer(model_url,input_shape=(224,224,3),trainable=False)
# model = tf.keras.Sequential([
#   tl_layer,
#   tf.keras.layers.Dense(len(os.listdir('./test')),activation='softmax')
# ])
# model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'],optimizer=tf.keras.optimizers.Adam())
# model.fit(train_data_small,validation_data=(valid_data),epochs=25,callbacks=[WandbCallback()])

In [40]:
# wandb.init(project=project_name,name='efficentnet-b5')
# model_url = "https://tfhub.dev/tensorflow/efficientnet/b5/feature-vector/1"
# tl_layer = hub.KerasLayer(model_url,input_shape=(224,224,3),trainable=False)
# model = tf.keras.Sequential([
#   tl_layer,
#   tf.keras.layers.Dense(len(os.listdir('./test')),activation='softmax')
# ])
# model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'],optimizer=tf.keras.optimizers.Adam())
# model.fit(train_data_small,validation_data=(valid_data),epochs=25,callbacks=[WandbCallback()])

In [41]:
# wandb.init(project=project_name,name='efficentnet-b6')
# model_url = "https://tfhub.dev/tensorflow/efficientnet/b6/feature-vector/1"
# tl_layer = hub.KerasLayer(model_url,input_shape=(224,224,3),trainable=False)
# model = tf.keras.Sequential([
#   tl_layer,
#   tf.keras.layers.Dense(len(os.listdir('./test')),activation='softmax')
# ])
# model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'],optimizer=tf.keras.optimizers.Adam())
# model.fit(train_data_small,validation_data=(valid_data),epochs=25,callbacks=[WandbCallback()])

In [42]:
# wandb.init(project=project_name,name='resnet50')
# model_url = "https://tfhub.dev/tensorflow/resnet_50/feature_vector/1"
# tl_layer = hub.KerasLayer(model_url,input_shape=(224,224,3),trainable=False)
# model = tf.keras.Sequential([
#   tl_layer,
#   tf.keras.layers.Dense(len(os.listdir('./test')),activation='softmax')
# ])
# model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'],optimizer=tf.keras.optimizers.Adam())
# model.fit(train_data_small,validation_data=(valid_data),epochs=25,callbacks=[WandbCallback()])

In [43]:
# wandb.init(project=project_name,name='efficentnet-b0')
# model_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"
# tl_layer = hub.KerasLayer(model_url,input_shape=(224,224,3),trainable=False)
# model = tf.keras.Sequential([
#   tl_layer,
#   tf.keras.layers.Dense(len(os.listdir('./test')),activation='softmax')
# ])
# model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'],optimizer=tf.keras.optimizers.Adam())
# model.fit(train_data_small,validation_data=(valid_data),epochs=25,callbacks=[WandbCallback()])

In [44]:
# wandb.init(project=project_name,name='efficentnet-b7')
# model_url = "https://tfhub.dev/tensorflow/efficientnet/b7/feature-vector/1"
# tl_layer = hub.KerasLayer(model_url,input_shape=(224,224,3),trainable=False)
# model = tf.keras.Sequential([
#   tl_layer,
#   tf.keras.layers.Dense(len(os.listdir('./test')),activation='softmax')
# ])
# model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'],optimizer=tf.keras.optimizers.Adam())
# model.fit(train_data_small,validation_data=(valid_data),epochs=25,callbacks=[WandbCallback()])

In [45]:
# wandb.init(project=project_name,name='mobilenet_v3_large_100_224')
# model_url = "https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5"
# tl_layer = hub.KerasLayer(model_url,input_shape=(224,224,3),trainable=False)
# model = tf.keras.Sequential([
#   tl_layer,
#   tf.keras.layers.Dense(len(os.listdir('./test')),activation='softmax')
# ])
# model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'],optimizer=tf.keras.optimizers.Adam())
# model.fit(train_data_small,validation_data=(valid_data),epochs=25,callbacks=[WandbCallback()])

# Results

- `lr` = 0.001
- `optimizer` = Adam()
- `epochs` = 250 (EarlyStopping = 12)
- `best model` = 

# Training the big dog model

In [48]:
wandb.init(project=project_name,name='Big-Dog-Model-0')
model_url = "https://tfhub.dev/tensorflow/resnet_50/feature_vector/1"
tl_layer = hub.KerasLayer(model_url,input_shape=(224,224,3),trainable=False)
model = tf.keras.Sequential([
  tl_layer,
  tf.keras.layers.Dense(len(os.listdir('./test')),activation='softmax')
])
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'],optimizer=tf.keras.optimizers.Adam(lr=0.001))
model.fit(train_data,validation_data=(valid_data),epochs=25,callbacks=[WandbCallback()])

epoch,8
loss,0.32252
accuracy,0.89724
val_loss,0.2489
val_accuracy,0.92877
_runtime,1176
_timestamp,1617527396
_step,8
best_val_loss,0.17482
best_epoch,6


epoch,▁▂▃▄▅▅▆▇█
loss,█▃▃▂▂▁▁▁▁
accuracy,▁▅▆▇▇▇███
val_loss,█▅▃▃▂▂▁▁▃
val_accuracy,▁▄▆▅▆▇█▇▇
_runtime,▁▂▃▄▅▅▆▇█
_timestamp,▁▂▃▄▅▅▆▇█
_step,▁▂▃▄▅▅▆▇█


Epoch 1/25
112/112 [==============================] - 128s 1s/step - loss: 2.6749 - accuracy: 0.3225 - val_loss: 0.5405 - val_accuracy: 0.8376
Epoch 2/25
112/112 [==============================] - 122s 1s/step - loss: 0.9025 - accuracy: 0.7375 - val_loss: 0.3646 - val_accuracy: 0.8832
Epoch 3/25
112/112 [==============================] - 123s 1s/step - loss: 0.6855 - accuracy: 0.7866 - val_loss: 0.3139 - val_accuracy: 0.9117
Epoch 4/25
112/112 [==============================] - 123s 1s/step - loss: 0.5608 - accuracy: 0.8304 - val_loss: 0.2909 - val_accuracy: 0.9202
Epoch 5/25
112/112 [==============================] - 125s 1s/step - loss: 0.4566 - accuracy: 0.8603 - val_loss: 0.2447 - val_accuracy: 0.9202
Epoch 6/25
112/112 [==============================] - 124s 1s/step - loss: 0.4403 - accuracy: 0.8629 - val_loss: 0.1856 - val_accuracy: 0.9402
Epoch 7/25
112/112 [==============================] - 124s 1s/step - loss: 0.3779 - accuracy: 0.8865 - val_loss: 0.2288 - val_accuracy: 0.9202

In [49]:
model.save('./big-dog-model-0.h5')

# Make predictions on test imgs

In [50]:
len(os.listdir('./test'))

36

# Done